In [ ]:
import re
import os, csv, sys 
import scipy
import pandas as pd
import gzip
import xml.etree.ElementTree as ET
from os import listdir
from datetime import datetime
import csv
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from dateutil import rrule
from datetime import datetime, timedelta
import requests
from pathlib import Path


In [ ]:
def hour_rounder(t):
    # Rounds to nearest hour by adding a timedelta hour if minute >= 30
    return (t.replace(second=0, microsecond=0, minute=0, hour=t.hour) - timedelta(hours=t.hour-6))

def xml2csvAll(filename,save_dir):
    print(f"{filename}")
    dateObj=os.path.basename(filename).split('_')[4]  
    forecastTimeObj=datetime.strptime(dateObj,"%Y%m%d%H%M%S")
    dfs=[]
    
    try:
        tree = ET.parse(filename)
    except ET.ParseError:
        print("Error with file, skipping")
        return
    root = tree.getroot()

    prod_center=root.find('header/productionCenter').text
    model_name=root.find('header/generatingApplication/model/name').text
    baseTime=root.find('header/baseTime').text
    print(model_name)

    ## Create one dictonary for each time point, and append it to a list
    for members in root.findall('data'):
        mtype=members.get('type')
        if mtype not in ['forecast', 'ensembleForecast']:
            continue
        for members2 in members.findall('disturbance'):
            
            cyclone_ID=members2.get('ID')      
            print(f"Found typhoon {cyclone_ID}")
            cyclone_name = [name.text.lower().strip() for name in members2.findall('cycloneName')]
            basin_name = [name.text.lower().strip() for name in members2.findall('basin')]
        
            if 'wp' not in basin_name:
                continue
            cyclone_name = cyclone_name[0].lower()
            print(f"Found typhoon {cyclone_name}")
            for members3 in members2.findall('fix'):
                tem_dic = {}
                time = [name.text for name in members3.findall('validTime')]
                #tem_dic['mtype']=[mtype]
                tem_dic['model_name']=[model_name]
                tem_dic['cyclone_ID']=[cyclone_ID]
                tem_dic['forecastTime']=[forecastTimeObj]
                #tem_dic['product']=[re.sub('\s+',' ',prod_center).strip().lower()]
                #tem_dic['cyc_number'] = [name.text for name in members2.findall('cycloneNumber')]
                tem_dic['ensemble']=[members.get('member')]
                tem_dic['speed'] = [name.text for name in members3.findall('cycloneData/maximumWind/speed')]
                tem_dic['pressure'] = [name.text for name in members3.findall('cycloneData/minimumPressure/pressure')]              
                tem_dic['time'] = ['/'.join(time[0].split('T')[0].split('-'))+', '+time[0].split('T')[1][:-1]]
                tem_dic['lat'] = [name.text for name in members3.findall('latitude')]
                tem_dic['lon']= [name.text for name in members3.findall('longitude')]
                tem_dic['lead_time']=[members3.get('hour')]
                #tem_dic['forecast_time'] = ['/'.join(baseTime.split('T')[0].split('-'))+', '+baseTime.split('T')[1][:-1]]
                tem_dic1 = dict( [(k,''.join(str(e).lower().strip() for e in v)) for k,v in tem_dic.items()])
                # Save to CSV
                dfs.append(pd.DataFrame(tem_dic1, index=[0]))
    fn=os.path.basename(filename)
    outfile = save_dir / f"csv/{fn}.csv"
    concatenated_df = pd.concat(dfs, ignore_index=True)
    concatenated_df.to_csv(outfile, mode='a', header=not os.path.exists(outfile), index=False)
    return concatenated_df


In [ ]:
## Parameters

data_folderxml = 'data/xml/'
data_foldercsv = 'data/csv/'
results_folder = 'results/'
figures_folder = 'figures/'

delete_previous_results_files = 'y'
save_csv_files = 'y'  # one per model

# Create folders

for folder_name in [data_folderxml, data_foldercsv,results_folder, figures_folder]:
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

In [ ]:
tem_dir='data/'

 
save_dir=os.path.join(tem_dir,'xml')
save_dir = Path(save_dir)

email = 'ateklesadik@redcross.nl'
pswd = 'yDrasYcB'

values = {'email' : email, 'passwd' : pswd, 'action' : 'login'}
login_url = 'https://rda.ucar.edu/cgi-bin/login'

ret = requests.post(login_url, data=values)


values = {'email' : email, 'passwd' : pswd, 'action' : 'login'}
login_url = 'https://rda.ucar.edu/cgi-bin/login'

ret = requests.post(login_url, data=values)
if ret.status_code != 200:
    print('Bad Authentication')
    print(ret.text)
    exit(1)

dspath = 'https://rda.ucar.edu/data/ds330.3/' #https://rda.ucar.edu/datasets/ds330.3/

verbose = True


tem_dir=os.getcwd()

save_dir=os.path.join(tem_dir,'data')
save_dir = Path(save_dir)

current_date = datetime.utcnow()
current_date=hour_rounder(current_date)
end_date = current_date
start_date =current_date - timedelta(hours=12)

date_list = rrule.rrule(rrule.HOURLY, 
                        dtstart=start_date, 
                        until=end_date,
                        interval=6)

 
forecaster='kwbc'
modelList={'CENS':'esttr',
           'CMC':'sttr',
           'GEFS':'esttr',
           'GFS':'sttr'}
for key, value in modelList.items():
    model=key
    modelcode=value
    for date in date_list:
        ymd = date.strftime("%Y%m%d")
        ymdhms = date.strftime("%Y%m%d%H%M%S")
        server = "test" if date < datetime(2008, 8, 1) else "prod"
        file = f'{forecaster}/{date.year}/{ymd}/z_tigge_c_{forecaster}_{ymdhms}_{model}_glob_{server}_{modelcode}_glo.xml'
        filename = 'https://data.rda.ucar.edu/ds330.3/' + file
        outfile = save_dir / "xml" / os.path.basename(filename)
        # Don't download if exists already
        if outfile.exists():
            if verbose:
                print(f'{file} already exists')
            continue
        req = requests.get(filename, cookies = ret.cookies, allow_redirects=True)
        if req.status_code != 200:
            if verbose:
                print(f'{file} invalid URL')
            continue
        if verbose:
            print(f'{file} downloading')
        open(outfile, 'wb').write(req.content)


# Get list of filenames
filename_list = sorted(list(Path(save_dir / "xml").glob('*.xml')))
for filename in filename_list[-4:]:
    xml2csvAll(filename,save_dir)

filename_list = sorted(list(Path(save_dir / "csv").glob('*.csv')))
dfs=[pd.read_csv(f) for f in filename_list]
concatenated_df = pd.concat(dfs, ignore_index=True)
